In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download -d shubhamgoel27/dermnet

100% 1.72G/1.72G [00:18<00:00, 141MB/s]
100% 1.72G/1.72G [00:18<00:00, 102MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dermnet.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Convolution2D, ZeroPadding2D,MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import tensorflow as tf
import keras
import pickle

In [ ]:
test_list=os.listdir('/content/test')
#len test_list
train_list=os.listdir('/content/train')
#len(train_list) 23

#SORTING THEM FOR REFERENCE
train_list.sort()
test_list.sort()
#viewing items
#test_list[0:5]

print(train_list)

In [ ]:
train_list_mod=['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Eczema Photos', 'Nail Fungus and other Nail Disease', 'Psoriasis pictures Lichen Planus and related diseases']

In [ ]:
len(train_list_mod)

In [ ]:
def data_dictionary():
    path_train="/content/train/"
    path_test="/content/test/"
    list_train=train_list_mod#os.listdir(path_train)
    train_dictionary={"image_path":[],"target":[]}
    test_dictionary={"image_path":[],"target":[]}
    k=0
    for i in list_train:
        path_disease_train=path_train+i
        path_disease_test=path_test+i
        image_list_train=os.listdir(path_disease_train)
        image_list_test=os.listdir(path_disease_test)
        for j in image_list_train:
            img_path_train=path_disease_train+"/"+j
            train_dictionary["image_path"].append(img_path_train)
            train_dictionary['target'].append(k)
        for m in image_list_test :
            img_path_test=path_disease_test+"/"+m
            test_dictionary["image_path"].append(img_path_test)
            test_dictionary['target'].append(k)
        k+=1
    test_df=pd.DataFrame(test_dictionary)
    train_df=pd.DataFrame(train_dictionary)

    return  train_df,test_df

In [ ]:
img=Image.open("/content/07Rhinophyma1.jpg")
plt.imshow(img)

In [ ]:
def load_data(input_size=(100,100)):
    images=[]
    images2=[]
    train_df,test_df=data_dictionary()
    for i in train_df['image_path']:
        img=cv2.imread(i)
        img=cv2.resize(img,input_size)
        images.append(img)
    y_train=np.asarray(train_df['target'])
    x_train=np.asarray(images)
    for i in test_df['image_path']:
        img=cv2.imread(i)
        img=cv2.resize(img,input_size)
        images2.append(img)
    y_test=np.asarray(test_df['target'])
    x_test=np.asarray(images2)
    return x_train,x_test,y_train,y_test



In [ ]:
x_train,x_test,y_train,y_test=load_data(input_size=(100,100))

In [ ]:
from tensorflow.keras.applications import  VGG19
from tensorflow.keras.models import Model
from keras.applications.vgg19 import preprocess_input


In [ ]:
vgg16 = VGG19(include_top=False,weights='imagenet')

In [ ]:
vgg16.summary()


In [ ]:
import joblib

In [ ]:
filename="vgg16.sav"
joblib.dump(vgg16,open(filename,'wb'))

In [ ]:
vgg19=joblib.load("vgg16.sav")

In [ ]:
train_img=preprocess_input(x_train)
test_img=preprocess_input(x_test)

In [ ]:
features_train=vgg16.predict(train_img)

In [ ]:
features_test=vgg16.predict(test_img)

In [ ]:
features_train.shape

In [ ]:
features_test.shape

In [ ]:
num_train=x_train.shape[0]
num_test=x_test.shape[0]


In [ ]:
x_train=features_train.reshape(num_train,4608)
x_test=features_test.reshape(num_test,4608)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten, Dense

In [ ]:
model=Sequential([ Dense(1024,activation='relu'),
                   Dense(512,activation='relu'),
                  Dense(256,activation='relu'),
                  Dense(128,activation='relu'),
                  Dense(6,activation='sigmoid'),

])

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=50,validation_data=(x_test,y_test))

In [ ]:
plt.plot(model.history.history['accuracy'],label="train_accuracy")
plt.plot(model.history.history['val_accuracy'],label="validation_accuracy")
plt.legend()
plt.show()


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy of CNN model: {:.2f}%'.format(accuracy * 100))

# Store accuracy in a variable
cnn_accuracy = accuracy

In [ ]:

def load_img(img_path):
    images=[]
    img=cv2.imread(img_path)
    img=cv2.resize(img,(100,100))
    images.append(img)
    x_test=np.asarray(images)
    test_img=preprocess_input(x_test)
    features_test=vgg16.predict(test_img)
    num_test=x_test.shape[0]
    f_img=features_test.reshape(num_test,4608)

    return f_img


In [ ]:
img=load_img("/content/test/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-20.jpg")

In [ ]:
np.argmax(model.predict(img))

In [ ]:
import pickle

In [ ]:
!mkdir -p saved_model
model.save('saved_model/skin_model')

In [ ]:
#filename="skin_model.pkl"
#joblib.dump(model,open(filename,'wb'))

In [ ]:
new_model = tf.keras.models.load_model('saved_model/skin_model')

# Check its architecture
new_model.summary()

In [ ]:
np.argmax(new_model.predict(img))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
nb = GaussianNB()

train_accs = []
test_accs = []

for i in range(10):
    nb.partial_fit(x_train, y_train, np.unique(y))
    train_accs.append(accuracy_score(y_train, nb.predict(x_train)))
    test_accs.append(accuracy_score(y_test, nb.predict(x_test)))

# Print the accuracy values
print("Training accuracies:", train_accs)
print("Testing accuracies:", test_accs)

# Plot the accuracy values
plt.plot(range(10), train_accs, label="Training")
plt.plot(range(10), test_accs, label="Testing")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()







In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

# Predict on training and testing sets
train_preds = lr.predict(x_train)
test_preds = lr.predict(x_test)

# Calculate accuracy on training and testing sets
train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)

# Print the accuracy values
print("Training accuracy:", train_acc)
print("Testing accuracy:", test_acc)

fig, ax = plt.subplots()
ax.bar(['Training Accuracy', 'Testing Accuracy'], [train_acc, test_acc])
ax.set_ylim([0, 1])
ax.set_ylabel('Accuracy')
ax.set_title('Logistic Regression Model Accuracy')
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(x_train, y_train)

# Predict on training and testing sets
train_preds = svm.predict(x_train)
test_preds = svm.predict(x_test)

# Calculate accuracy on training and testing sets
train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)

# Print the accuracy values
print("Training accuracy:", train_acc)
print("Testing accuracy:", test_acc)

acc_values = [train_acc, test_acc]
acc_names = ['Training', 'Testing']
plt.bar(acc_names, acc_values)
plt.title('Accuracy of svm')
plt.xlabel('Data Split')
plt.ylabel('Accuracy')
plt.ylim((0.0, 1.0))
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

# Predict on training and testing sets
train_preds = rf.predict(x_train)
test_preds = rf.predict(x_test)

# Calculate accuracy on training and testing sets
train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, test_preds)

# Print the accuracy values
print("Training accuracy:", train_acc)
print("Testing accuracy:", test_acc)

acc_values = [train_acc, test_acc]
acc_names = ['Training', 'Testing']
plt.bar(acc_names, acc_values)
plt.title('Accuracy of Random Forest Model')
plt.xlabel('Data Split')
plt.ylabel('Accuracy')
plt.ylim((0.0, 1.0))
plt.show()

confusion matrix for all above


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Train Logistic Regression model
lr = LogisticRegression()
lr.fit(x_train, y_train)

# Predict on test set
y_pred_lr = lr.predict(x_test)

# Create confusion matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
accuracy = accuracy_score(y_test, y_pred_lr)
err_rate = 1 - accuracy

print("Accuracy:", accuracy)
print('Error rate:', err_rate)
# Plot confusion matrix using seaborn heatmap
sns.heatmap(cm_lr, annot=True, cmap='Blues')
plt.title('Confusion Matrix for Logistic Regression')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
# Train Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(x_test)

# Create confusion matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)

accuracy = accuracy_score(y_test, y_pred_rf)
err_rate = 1 - accuracy

print("Accuracy:", accuracy)
print('Error rate:', err_rate)

# Plot confusion matrix using seaborn heatmap
sns.heatmap(cm_rf, annot=True, cmap='Blues')
plt.title('Confusion Matrix for Random Forest')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
# Train Kernel SVM model
svm = SVC(kernel='rbf', random_state=42)
svm.fit(x_train, y_train)

# Predict on test set
y_pred_svm = svm.predict(x_test)

# Create confusion matrix
cm_svm = confusion_matrix(y_test, y_pred_svm)
accuracy = accuracy_score(y_test, y_pred_svm)
err_rate = 1 - accuracy

print("Accuracy:", accuracy)
print('Error rate:', err_rate)

# Plot confusion matrix using seaborn heatmap5
sns.heatmap(cm_svm, annot=True, cmap='Blues')
plt.title('Confusion Matrix for Kernel SVM')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
# Train Naive Bayes model
nb = GaussianNB()
nb.fit(x_train, y_train)

# Predict on test set
y_pred_nb = nb.predict(x_test)

# Create confusion matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
accuracy = accuracy_score(y_test, y_pred_nb)
err_rate = 1 - accuracy

print("Accuracy:", accuracy)
print('Error rate:', err_rate)

# Plot confusion matrix using seaborn heatmap
sns.heatmap(cm_nb, annot=True, cmap='Blues')
plt.title('Confusion Matrix for Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Train and predict using all models
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)
acc_lr = accuracy_score(y_test, y_pred_lr)

svm = SVC(kernel='rbf', random_state=42)
svm.fit(x_train, y_train)
y_pred_svm = svm.predict(x_test)
acc_svm = accuracy_score(y_test, y_pred_svm)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
acc_rf = accuracy_score(y_test, y_pred_rf)

nb = GaussianNB()
nb.fit(x_train, y_train)
y_pred_nb = nb.predict(x_test)
acc_nb = accuracy_score(y_test, y_pred_nb)

# Create and plot accuracy bar chart
accuracy_scores = [acc_lr, acc_svm, acc_rf, acc_nb, cnn_accuracy]
model_names = ['Logistic Regression', 'Kernel SVM', 'Random Forest', 'Naive Bayes', 'CNN']
plt.plot(model_names, accuracy_scores, marker='o',markerfacecolor='black', color="black")
plt.title('Accuracy of Classification Algorithms')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.show()